In [1]:
import rosbag
import cv2
from cv_bridge import CvBridge
import os

from geometry_msgs.msg import Twist
from sensor_msgs.msg import Image
from bebop_msgs.msg import Ardrone3PilotingStateAltitudeChanged

In [2]:
repoRoot = r'.'

path = 'test'
if not os.path.exists(path):
    os.makedirs(path)

In [4]:
bag = rosbag.Bag('test_2020-06-15-19-36-39.bag')
bridge = CvBridge()

cnt = 0
for topic, msg, t in bag.read_messages(topics=['/bebop/cmd_vel', '/bebop/image_raw', '/flow']):
    if topic == '/flow':
        cvImage = bridge.imgmsg_to_cv2(msg, 'bgr8')
        out_name = os.path.join(repoRoot, path, '%06d.png' % cnt)
        cv2.imwrite(out_name, cvImage) 
        cnt = cnt + 1
bag.close()

In [11]:
import yaml
from rosbag.bag import Bag

info_dict = yaml.load(Bag('test_2020-06-15-19-36-39.bag', 'r')._get_yaml_info())
print(info_dict)

{'end': 1592267819.695702, 'compression': 'none', 'topics': [{'topic': '/bebop/cmd_vel', 'type': 'geometry_msgs/Twist', 'frequency': 29.9972, 'messages': 552}, {'topic': '/bebop/image_raw', 'type': 'sensor_msgs/Image', 'frequency': 30.4752, 'messages': 519}, {'topic': '/bebop/states/ardrone3/PilotingState/AltitudeChanged', 'type': 'bebop_msgs/Ardrone3PilotingStateAltitudeChanged', 'frequency': 4.9792, 'messages': 98}, {'topic': '/flow', 'type': 'sensor_msgs/Image', 'frequency': 11.1553, 'messages': 136}], 'messages': 1305, 'start': 1592267800.078959, 'version': 2.0, 'types': [{'type': 'bebop_msgs/Ardrone3PilotingStateAltitudeChanged', 'md5': '5073f650d09c8192d358641b48a0204b'}, {'type': 'geometry_msgs/Twist', 'md5': '9f195f881246fdfa2798d1d3eebca84a'}, {'type': 'sensor_msgs/Image', 'md5': '060021388200f6f0f447d0fcd9c64743'}], 'indexed': True, 'path': 'test_2020-06-15-19-36-39.bag', 'duration': 19.616743, 'size': 559156142}


/home/esteban/Code/AeroFlowControl/venv/lib/python2.7/site-packages/ipykernel_launcher.py:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  after removing the cwd from sys.path.
